기존에 cgv에서 영화 정보를 가져온 것(검색이 가능한 것)들 목록을 통해 줄거리를 추가 저장한다.  
즉, cgv_전처리_통합파일.csv를 기반으로 진행한다

In [23]:
import pandas as pd
movie_list=pd.read_csv('../cgv리뷰_전처리/전처리파일저장/cgv_전처리_통합파일.csv', encoding='utf-8-sig')
movie_list_df=movie_list[["영화제목"]].copy()
movie_list_df['인덱스']=movie_list_df.index+1
movie_list_df.head()

,영화제목,인덱스
0,매트릭스3 레볼루션,1
1,써클,2
2,폴리와 함께,3
3,시월애,4
4,보리밭을 흔드는 바람,5


In [24]:
# 필요한 라이브러리들을 가져옵니다.
import pandas as pd
import os
import re
import time
from tqdm import tqdm # 진행 상황을 시각적으로 보여주기 위한 라이브러리

# 셀레니움 관련 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# 웹드라이버 자동 설치를 위한 라이브-러리
from webdriver_manager.chrome import ChromeDriverManager

# --- 설정 ---
# 몇 번째 영화부터 크롤링을 시작할지 CSV의 '인덱스' 기준으로 지정합니다. (기본값: 1)
# 예를 들어 11로 설정하면, 인덱스 1~10번 영화는 건너뛰고 11번 영화부터 시작합니다.
START_FROM_INDEX = 1

# --- 0. 크롬 드라이버 설정 ---
# GoogleBot으로 User-Agent를 설정합니다.
chrome_options = Options()
chrome_options.add_argument("user-agent=GoogleBot")
# 백그라운드에서 실행하려면 아래 주석을 해제하세요.
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level=3") # 콘솔 로그를 최소화합니다.
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬 드라이버를 최신 버전으로 자동 설치하고 서비스를 시작합니다.
try:
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # 웹 페이지의 요소가 나타날 때까지 최대 10초간 기다리는 WebDriverWait 객체를 생성합니다.
    wait = WebDriverWait(driver, 10)
    print("✅ 웹 드라이버 설정이 완료되었습니다.")
except Exception as e:
    print(f"❌ 웹 드라이버 설정 중 오류가 발생했습니다: {e}")
    print("인터넷 연결을 확인하거나, Chrome 브라우저가 설치되어 있는지 확인해주세요.")
    driver = None # 드라이버 설정 실패 시 이후 코드 실행 방지

# --- 1. 크롤링할 영화 목록 불러오기 ---
if driver:
    try:
        # 파일 경로는 스크립트 실행 위치에 맞게 조정해야 할 수 있습니다.
        movie_list=pd.read_csv('../cgv리뷰_전처리/전처리파일저장/cgv_전처리_통합파일.csv', encoding='utf-8-sig')
        movie_list_df=movie_list[["영화제목"]].copy()
        movie_list_df['인덱스']=movie_list_df.index+1
        # '인덱스'와 '영화제목' 컬럼이 있는지 확인
        if '인덱스' not in movie_list_df.columns or '영화제목' not in movie_list_df.columns:
            raise ValueError("'인덱스' 또는 '영화제목' 컬럼을 찾을 수 없습니다. CSV 파일을 확인해주세요.")
        print(f"✅ 총 {len(movie_list_df)}개의 영화 목록을 불러왔습니다.")
    except FileNotFoundError:
        print("❌ '영화목록.csv' 파일을 찾을 수 없습니다. 파일 경로(../../영화관람객크롤링/영화목록.csv)를 확인해주세요.")
        movie_list_df = None # 파일이 없으면 None으로 초기화하여 에러 방지
    except ValueError as e:
        print(f"❌ CSV 파일 컬럼 오류: {e}")
        movie_list_df = None

# --- 2. 결과 저장 폴더 생성 ---
if driver and movie_list_df is not None:
    output_dir = "../영화리뷰데이터_저장/cgv_장르_줄거리"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"✅ '{output_dir}' 폴더를 생성했습니다.")

# --- 메인 크롤링 로직 ---
if driver and movie_list_df is not None:
    # iterrows()를 사용하여 데이터프레임을 한 줄씩 순회합니다.
    for _, row in tqdm(movie_list_df.iterrows(), total=movie_list_df.shape[0], desc="전체 영화 크롤링 진행률"):
        movie_index = row['인덱스']
        movie_title = row['영화제목']

        # 시작 인덱스 체크
        if movie_index < START_FROM_INDEX:
            continue # 시작 인덱스보다 작으면 현재 영화는 건너뜁니다.

        print(f"\n[{movie_index}/{len(movie_list_df)}] '{movie_title}' 크롤링 시작...")

        # 크롤링한 데이터를 저장할 변수 초기화
        movie_data = {}
        genre_text = pd.NA
        plot_text = pd.NA

        try:
            # 3. CGV 영화 검색 페이지로 이동
            driver.get("https://cgv.co.kr/tme/itgrSrch")

            # 4. 검색어 입력
            search_box = wait.until(EC.presence_of_element_located((By.ID, "swrd")))
            search_box.clear()
            search_box.send_keys(movie_title)
            
            # 로딩 오버레이가 사라질 때까지 기다립니다.
            try:
                loading_overlay_selector = "div.loading_pageContainer__fvLY_"
                wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, loading_overlay_selector)))
            except TimeoutException:
                pass # 로딩 오버레이가 없으면 정상 진행

            # 5. 검색 버튼 클릭
            search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-sch")))
            search_button.click()

            # 6. '영화' 탭 클릭
            try:
                short_wait = WebDriverWait(driver, 2)
                movie_tab = short_wait.until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='영화']]")))
                movie_tab.click()
            except TimeoutException:
                print(f"-> '{movie_title}' 검색 결과에서 '영화' 탭을 찾을 수 없습니다. 다음 영화로 넘어갑니다.")
                time.sleep(0.5)
                continue

            # 7. 첫 번째 검색 결과 클릭
            try:
                first_movie_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.tmes01040_section__D67K1 ul > li:first-child > button")))
                first_movie_link.click()
            except TimeoutException:
                print(f"-> '{movie_title}' 영화 목록을 찾을 수 없습니다. 다음 영화로 넘어갑니다.")
                continue

            # --- 영화 상세 페이지 데이터 추출 ---

            # 8. 영화 장르 추출
            try:
                short_wait = WebDriverWait(driver, 0.5)
                genre_selector = '.cnms01020_textInfoWrap__M5Cuo > span:nth-child(3)'
                genre_element = short_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, genre_selector)))
                genre_text = genre_element.text.strip()
                print(f"-> 장르: {genre_text}")
            except TimeoutException:
                print("-> 장르 정보를 찾을 수 없습니다.")

            # 9. 영화 줄거리 추출
            try:
                short_wait = WebDriverWait(driver, 0.5)
                plot_selector = '.cnms01020_description__7nj95'
                plot_element = short_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, plot_selector)))
                # .text 속성은 <br> 태그를 포함한 줄바꿈을 자동으로 처리해줍니다.
                plot_text = plot_element.text.strip()
                print(f"-> 줄거리 추출 완료 (길이: {len(plot_text)})")
            except TimeoutException:
                print("-> 줄거리 정보를 찾을 수 없습니다.")

            # # [수정] 장르 또는 줄거리 정보가 없으면 다음 영화로 진행 (불필요한 time.sleep 제거)
            # if pd.isna(genre_text) or pd.isna(plot_text):
            #     print(f"-> 장르 또는 줄거리 정보가 없어 '{movie_title}' 크롤링을 건너뜁니다.")
            #     continue

            # 10. 수집한 데이터를 딕셔너리에 저장
            movie_data = {
                '영화제목': movie_title,
                '장르': genre_text,
                '줄거리': plot_text
            }

        except Exception as e:
            print(f"❌ '{movie_title}' 크롤링 중 예상치 못한 오류 발생: {e}")
            continue

        # --- 11. 추출한 데이터를 CSV 파일로 저장 ---
        if movie_data:
            df = pd.DataFrame([movie_data])
            # 파일 이름에 인덱스를 사용하여 각 영화 데이터를 개별 파일로 저장
            file_path = os.path.join(output_dir, f"{movie_index}.csv")
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"✅ '{movie_title}' 데이터 저장 완료: {file_path}")
        else:
            print(f"-> '{movie_title}'에 대한 데이터를 저장하지 못했습니다.")

    # 모든 작업이 끝나면 드라이버 종료
    if driver:
        driver.quit()
    print("\n🎉 모든 영화에 대한 크롤링 작업이 완료되었습니다.")



✅ 웹 드라이버 설정이 완료되었습니다.
✅ 총 7001개의 영화 목록을 불러왔습니다.


전체 영화 크롤링 진행률:   0%|          | 0/7001 [00:00<?, ?it/s]


[1/7001] '매트릭스3 레볼루션' 크롤링 시작...
-> 장르 정보를 찾을 수 없습니다.


전체 영화 크롤링 진행률:   0%|          | 1/7001 [00:04<8:46:24,  4.51s/it]

-> 줄거리 추출 완료 (길이: 1298)
✅ '매트릭스3 레볼루션' 데이터 저장 완료: ../영화리뷰데이터_저장/cgv_장르_줄거리/1.csv

[2/7001] '써클' 크롤링 시작...


...

전체 영화 크롤링 진행률: 100%|█████████▉| 7000/7001 [6:44:20<00:03,  3.42s/it]

-> 장르: 애니메이션
-> 줄거리 추출 완료 (길이: 782)
✅ '구하라!' 데이터 저장 완료: ../영화리뷰데이터_저장/cgv_장르_줄거리/7000.csv

[7001/7001] '톤레삽강은 멈추지 않는다' 크롤링 시작...


전체 영화 크롤링 진행률: 100%|██████████| 7001/7001 [6:44:24<00:00,  3.47s/it]

-> 장르: 다큐멘터리
-> 줄거리 추출 완료 (길이: 470)
✅ '톤레삽강은 멈추지 않는다' 데이터 저장 완료: ../영화리뷰데이터_저장/cgv_장르_줄거리/7001.csv



🎉 모든 영화에 대한 크롤링 작업이 완료되었습니다.
